### Import some modules

In [1]:
import cv2
import time

# Set up Camera access(es) ###
Check possible device ids using: ```ls -ltrh /dev/video*```

In [2]:
devices = {}
for i in range(10):
    success = cv2.VideoCapture(i).read()[0]
    if success:
        name = 'cam ' + str(i)
        devices[name] = cv2.VideoCapture(i)
print(len(devices.keys()), 'video devices have been initialised.' )

1 video devices have been initialised.


### Define Live Preview Window ###
Initiate the necessary modules

In [3]:
%gui qt5
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pyqtgraph

# and some modules for button widgets
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

ImportError: cannot import name 'QtGui'

--- Logging error ---
Traceback (most recent call last):
  File "/home/pi/berryconda3/lib/python3.6/site-packages/tornado/ioloop.py", line 605, in _run_callback
    ret = callback()
  File "/home/pi/berryconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/pi/berryconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 263, in enter_eventloop
    self.eventloop(self)
  File "/home/pi/berryconda3/lib/python3.6/site-packages/ipykernel/eventloops.py", line 123, in loop_qt5
    return loop_qt4(kernel)
  File "/home/pi/berryconda3/lib/python3.6/site-packages/ipykernel/eventloops.py", line 105, in loop_qt4
    kernel.app = get_app_qt4([" "])
  File "/home/pi/berryconda3/lib/python3.6/site-packages/IPython/lib/guisupport.py", line 114, in get_app_qt4
    from IPython.external.qt_for_kernel import QtGui
  File "/home/pi/berryconda3/lib/python3.6/site-packages/IPython/external/qt_for_kernel.py", line 95, i

Define a Preview **object class**

In [4]:
class Preview(object):   
    """Pop up a qt5 window for each working camera and stream the **live camera view** using pyqtgraph"""
    def __init__(self, device):
        
        # assign camera
        self.device = devices[device]
        self.device_name = device
        
        # define preview window
        self.window_title = "Preview " + self.device_name
        self.window = pyqtgraph.GraphicsWindow(title=self.window_title)
        self.window.resize(640,480)
        self.viewport = pyqtgraph.GraphicsLayout()
        self.view = self.viewport.addViewBox()
        self.window.setCentralItem(self.viewport)
        
        # init timer for data refresh
        self.stream = QtCore.QTimer()
        self.stream.timeout.connect(self.update)
        self.stream.start(10) # pass time in miliseconds
        
        # define button for saving images
        self.save_button = widgets.Button(description= self.device_name, button_style = '', icon='save')
        self.save_button.on_click(self.save)
        
        
    def update(self):
        bgr_img = [self.device.read() for i in range(7)][-1][1]
        b,g,r = cv2.split(bgr_img)
        rgb_img = cv2.merge([r,g,b])

        frame = pyqtgraph.ImageItem(rgb_img)
        self.view.addItem(frame)
        
    def save(self, b):
        self.stream.stop()

        img = [self.device.read() for i in range(7)][-1][1]
        cv2.imwrite(self.device_name +'_'+time.asctime()+'.png', img)

        self.stream.start(10)
        self.window.show()   

**Note:** *Code for saving pictures explained*:
The camera is somewhat slow in data-acquisition, a changed view may not be reflected immediately in the picture. 
Therefore we send 7 read() requests and only store the last one (index [-1]) in the variable "img".
Each read() request returns a tuple of success (True, False) and the image data, the latter is written to a *".png"* file

# Live Preview#
Pop up a qt5 window for each working camera and stream the **live camera view** using pyqtgraph

In [5]:
preview = {}
for camera in devices.keys():    
    preview[camera] = Preview(camera)

# show the save-buttons of all devices    
display(widgets.HBox(
                    [preview[camera].save_button for camera in devices.keys()]
                    ))

A Jupyter Widget

## **End the streams** and close the pop-up windows

In [28]:
for cam in preview.keys():
    preview[cam].stream.stop()
    preview[stream].window.close()

# Start continuous picture acquisition#
Define period in minutes

In [6]:
period = .2

Start a **while loop**, which saves a picture for each camera, then waits for the specified period until next repetition. 
(* **Note**: repeats forever if you don't stop it, e.g. using the notebook's "interrupt Kernel" button above*)

In [1]:
while True:
    try: 
        for cam in preview.keys():

            img = [preview[cam].device.read() for i in range(7)][-1][1]
            cv2.imwrite(cam +'_'+ time.asctime()+'.png', img)
            
            time.sleep(period*60)  # pass period in seconds
            
    except KeyboardInterrupt:
        print('Process ended by user.')
        break

NameError: name 'preview' is not defined